#### Creation Of Knowledge Graph

This notebook will contain ALL the codes to create the knowledge graph. We will be defining nodes and relationships. 

#### Set up the Knowledge Graph

In [17]:
import os
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph

load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')

In [18]:
kg = Neo4jGraph(
    url = NEO4J_URI, username = NEO4J_USERNAME, password = NEO4J_PASSWORD, database = NEO4J_DATABASE
)

In [19]:
import logging

logging.basicConfig(
    filename='neo4j_import.log',
    level=logging.INFO,
    format='%(asctime)s %(levelname)s:%(message)s'
)

##### 1. Define Nodes
- Course
    - courseCode (string, unique identifier)
    - academicUnits (integer)
    - title (string)
    - description (string)
- Degree
    - degreeCode (string, unique identifier)
    - degree_name (string)
    - title (string)
    - degree_type (string)
    - description (string)
    - admission_requirements (string)
    - programme_duration (string)
    - career_prospects (string)
- School
    - schoolCode (string, unique identifier)
    - schoolName (string)
    - description (string)
- Type (of courses)
    - typeName (ICC, Core, MPE, BDE)

In [20]:
cypher = """
    CREATE CONSTRAINT FOR (c:Course) REQUIRE c.courseCode IS UNIQUE
"""

results = kg.query(cypher)

ClientError: {code: Neo.ClientError.Schema.EquivalentSchemaRuleAlreadyExists} {message: An equivalent constraint already exists, 'Constraint( id=3, name='constraint_428e5cbf', type='UNIQUENESS', schema=(:Course {courseCode}), ownedIndex=2 )'.}

In [ ]:
cypher = """
    CREATE CONSTRAINT FOR (c:Course) REQUIRE c.courseCode IS UNIQUE
"""
results = kg.query(cypher)

cypher = """
    CREATE CONSTRAINT FOR (d:Degree) REQUIRE d.degreeCode IS UNIQUE
"""

results = kg.query(cypher)

cypher = """
    CREATE CONSTRAINT FOR (s:School) REQUIRE s.schoolCode IS UNIQUE
"""

results = kg.query(cypher)

cypher ="""
        CREATE CONSTRAINT FOR (t:Type) REQUIRE t.typeName IS UNIQUE
        """

results = kg.query(cypher)


In [ ]:
import pandas as pd

# Helper function to load courses nodes into Neo4j graph database
def load_courses(neo4j_graph, filepath):
    df = pd.read_csv(filepath)
    for index, row in df.iterrows():
        cypher_query = """
        MERGE (c:Course {courseCode: '$courseCode'})
        SET c.academicUnits = toInteger($academicUnits),
            c.title = $title,
            c.description = $description
        """
        parameters = {
            "courseCode": row["courseCode"],
            "academicUnits": row["academicUnits"],
            "title": row["title"],
            "description": row["description"]
        }
        neo4j_graph.query(cypher_query, parameters)

In [ ]:
load_courses(kg, "data/courses_nodes.csv")

In [ ]:
# Helper function to load degree nodes into Neo4j graph database
def load_degrees(neo4j_graph, filepath):
    df = pd.read_csv(filepath)
    for index, row in df.iterrows():
        cypher_query = """
                       MERGE (d:Degree {degreeCode: $degreeCode})
                       SET d.degree_name = $degree_name,
                           d.title = $title,
                           d.degree_type = $degree_type,
                           d.description = $description, 
                           d.admission_requirements = $admission_requirements,
                           d.programme_duration = $programme_duration,
                           d.career_prospects = $career_prospects
                       """
        parameters = {
            "degreeCode": row["degreeCode"],
            "title": row["title"],
            "degree_name": row["degree_name"],
            "degree_type": row["degree_type"],
            "description": row["description"],
            "admission_requirements": row["admission_requirements"],
            "programme_duration": row["programme_duration"],
            "career_prospects": row["career_prospects"]
        }

        neo4j_graph.query(cypher_query, parameters)


In [ ]:
load_degrees(kg, "data/degree_nodes.csv")

In [ ]:
# Helper function to load school nodes into Neo4j graph database
def load_schools(neo4j_graph, filepath):
    df = pd.read_csv(filepath)
    for index, row in df.iterrows():
        cypher_query = """
                       MERGE (s:School {schoolCode: $schoolCode})
                       SET s.schoolName = $schoolName
                       SET s.description = $description
                       """
        parameters = {
            "schoolCode": row["schoolCode"],
            "schoolName": row["schoolName"],
            "description": row["description"]
        }

        neo4j_graph.query(cypher_query, parameters)
        

In [ ]:
load_schools(kg, "data/school_nodes.csv")

In [ ]:
# Helper function to load type nodes into Neo4j graph database
def load_types(neo4j_graph, filepath):
    df = pd.read_csv(filepath)
    for index, row in df.iterrows():
        cypher_query = """
                       MERGE (t:Type {typeName: $typeName})
                       """
        parameters = {
            "typeName": row["typeName"]
        }

        neo4j_graph.query(cypher_query, parameters)

In [ ]:
load_types(kg, "data/type_nodes.csv")

In [ ]:
# Helper function to load yearStanding nodes into Neo4j graph database
def load_yearStanding(neo4j_graph, filepath):
    df = pd.read_csv(filepath)
    for index, row in df.iterrows():
        cypher_query = """
                       MERGE (y:yearStanding {year: $yearStanding})
                       """
        parameters = {
            "yearStanding": row["yearStanding"]
        }

        neo4j_graph.query(cypher_query, parameters)

In [ ]:
load_yearStanding(kg, "data/yearStanding_nodes.csv")

In [ ]:
# Helper function to load semesterName nodes into Neo4j graph database
def load_semester(neo4j_graph, filepath):
    df = pd.read_csv(filepath)
    for index, row in df.iterrows():
        cypher_query = """
                       MERGE (s:Semester {semester: $semesterName})
                       """
        parameters = {
            "semesterName": row["semesterName"]
        }

        neo4j_graph.query(cypher_query, parameters)

In [ ]:
load_semester(kg, "data/semesterName_nodes.csv")

In [ ]:
# Helper function to load specialisation nodes into Neo4j graph database
def load_specialisation(neo4j_graph, filepath):
    df = pd.read_csv(filepath)
    for index, row in df.iterrows():
        cypher_query = """
                       MERGE (s:Specialisation {specialisation: $Specialisation})
                       """
        parameters = {
            "Specialisation": row["Specialisation"]
        }

        neo4j_graph.query(cypher_query, parameters)

In [ ]:
load_specialisation(kg, "data/specialisation_nodes.csv")

#### Create Relationships between nodes
- (course)-[:PRE_REQUISITE_FOR]->(course)
- (course)-[:CO_REQUISITE_FOR]->(course)
- (course)-[:BELONGS_TO_SCHOOL]->(school)
- (course)-[:HAS_TYPE]->(type)
- (course)-[:SCHEDULED_IN]->(semester)
- (course)-[:REQUIRED_FOR]->(degree)

##### 1. Pre_requisite Relationships between nodes
Prerequisite relationships in courses can vary: 
- Single Course Prerequisite: A course requires another specific course (e.g. SC1003)
- Multiple Courses (OR Condition): A course requires at least one from set of courses (e.g. SC1003 OR SC1007)
- Year Standing: A course requires completion of a certain year (e.g. Year 3)

We will need a schema that can represent both simple and compound conditions efficiently.

In [ ]:
# Add the pre_requisite relationship 
def link_prerequisite(neo4j_graph, filepath):
    df = pd.read_csv(filepath)

    for index, row in df.iterrows():
        if row["prerequisiteType"] == "Course":
            cypher_query = """
                           MATCH (pre: Course {courseCode: $prerequisiteValue})
                           MATCH (c: Course {courseCode: $courseCode})
                           MERGE (pre)-[:PRE_REQUISITE_FOR]->(c)
                           """
            parameters = {
               "prerequisiteValue": row["prerequisiteValue"],
               "courseCode": row["courseCode"]
            }

            try:
                neo4j_graph.query(cypher_query, parameters)
                logging.info(f"Linked Prerequisite Course {row['prerequisiteValue']} to Course {row['courseCode']}")
            except Exception as e:
                logging.error(f"Error linking Prerequisite Course {row['prerequisiteValue']} to Course {row['courseCode']}: {e}")

        elif row["prerequisiteType"] == "Year":
            cypher_query = """
                           MATCH (y:yearStanding {year: $yearStanding})
                           MATCH (c: Course {courseCode: $courseCode})
                           MERGE (y)-[:PRE_REQUISITE_FOR]->(c)
                           """
            parameters = {
                "yearStanding": row["prerequisiteValue"],
                "courseCode": row["courseCode"]
            }

            try:
                neo4j_graph.query(cypher_query, parameters)
                logging.info(f"Linked Year {row['prerequisiteValue']} as a prerequisite for Course {row['courseCode']}")
            except Exception as e:
                logging.error(f"Error linking Year {row['prerequisiteValue']} to Course {row['courseCode']}: {e}")
        
        elif row["prerequisiteType"] == "OR_Course":
            # Handles OR prerequisites by creating a Prerequisite Group
            # First, create of find a PrerequisiteGroup node with logicType 'OR'
            # Form the group_id
            group_id = f"{row['courseCode']}_OR_Group"

            cypher_group = """
                           MERGE (g: PrerequisiteGroup {groupId: $groupId, logicType: 'OR'})
                           """
            parameters_group = {
                "groupId": group_id
            }

            try:
                neo4j_graph.query(cypher_group, parameters_group)
                logging.info(f"Created/Found PrerequisiteGroup {group_id} with logicType 'OR'")
            except Exception as e:
                logging.error(f"Error creating PrerequisiteGroup {group_id}: {e}")

            # Link the group to the course
            cypher_course_link = """
                                 MATCH (c: Course {courseCode: $courseCode})
                                 MATCH (g: PrerequisiteGroup {groupId: $groupId})
                                 MERGE (g)-[:PRE_REQUISITE_FOR]->(c)
                                 """
            parameters_course_link = {
                "courseCode": row["courseCode"],
                "groupId": group_id
            }

            try:
                neo4j_graph.query(cypher_course_link, parameters_course_link)
                logging.info(f"Linked Course {row['courseCode']} to PrerequisiteGroup {group_id}")
            except Exception as e:
                logging.error(f"Error linking Course {row['courseCode']} to PrerequisiteGroup {group_id}: {e}")

            # Link the prerequisite course to the group
            cypher_requires = """
                              MATCH (pre: Course {courseCode: $prerequisiteCourseCode})
                              MATCH (g: PrerequisiteGroup {groupId: $groupId})
                              MERGE (g)-[:REQUIRES]->(pre)
                              """
            parameters_requires = {
                "prerequisiteCourseCode": row["prerequisiteValue"],
                "groupId": group_id
            }

            try:
                neo4j_graph.query(cypher_requires, parameters_requires)
                logging.info(f"Linked Prerequisite Course {row['prerequisiteValue']} to PrerequisiteGroup {group_id}")
            except Exception as e:
                logging.error(f"Error linking Prerequisite Course {row['prerequisiteValue']} to PrerequisiteGroup {group_id}: {e}")
            

           

In [12]:
link_prerequisite(kg, "data/pre_requisite_relationship.csv")

In [ ]:
def unlink_prerequisite(neo4j_graph, filepath):
    df = pd.read_csv(filepath)

    for index, row in df.iterrows():
        if row["prerequisiteType"] == "Course":
            # Delete PRE_REQUISITE_FOR relationship between Courses
            cypher_query = """
                           MATCH (pre:Course {courseCode: $prerequisiteValue})-[r:PRE_REQUISITE_FOR]->(c:Course {courseCode: $courseCode})
                           DELETE r
                           """
            parameters = {
               "prerequisiteValue": row["prerequisiteValue"],
               "courseCode": row["courseCode"]
            }

            try:
                neo4j_graph.query(cypher_query, parameters)
                logging.info(f"Unlinked Prerequisite Course {row['prerequisiteValue']} from Course {row['courseCode']}")
            except Exception as e:
                logging.error(f"Error unlinking Prerequisite Course {row['prerequisiteValue']} from Course {row['courseCode']}: {e}")

        elif row["prerequisiteType"] == "Year":
            # Delete PRE_REQUISITE_FOR relationship between YearStanding and Course
            cypher_query = """
                           MATCH (y:yearStanding {year: $yearStanding})-[r:PRE_REQUISITE_FOR]->(c:Course {courseCode: $courseCode})
                           DELETE r
                           """
            parameters = {
                "yearStanding": row["prerequisiteValue"],
                "courseCode": row["courseCode"]
            }

            try:
                neo4j_graph.query(cypher_query, parameters)
                logging.info(f"Unlinked Year {row['prerequisiteValue']} as a prerequisite from Course {row['courseCode']}")
            except Exception as e:
                logging.error(f"Error unlinking Year {row['prerequisiteValue']} from Course {row['courseCode']}: {e}")
        
        elif row["prerequisiteType"] == "OR_Course":
            # Remove relationships involving PrerequisiteGroup
            group_id = f"{row['courseCode']}_OR_Group"

            # Delete REQUIRES relationship between PrerequisiteGroup and Prerequisite Course
            cypher_requires = """
                              MATCH (g:PrerequisiteGroup {groupId: $groupId})-[r:REQUIRES]->(pre:Course {courseCode: $prerequisiteCourseCode})
                              DELETE r
                              """
            parameters_requires = {
                "prerequisiteCourseCode": row["prerequisiteValue"],
                "groupId": group_id
            }

            try:
                neo4j_graph.query(cypher_requires, parameters_requires)
                logging.info(f"Unlinked Prerequisite Course {row['prerequisiteValue']} from PrerequisiteGroup {group_id}")
            except Exception as e:
                logging.error(f"Error unlinking Prerequisite Course {row['prerequisiteValue']} from PrerequisiteGroup {group_id}: {e}")

            # Delete PRE_REQUISITE_FOR relationship between Course and PrerequisiteGroup
            cypher_course_link = """
                                 MATCH (c:Course {courseCode: $courseCode})-[r:PRE_REQUISITE_FOR]->(g:PrerequisiteGroup {groupId: $groupId})
                                 DELETE r
                                 """
            parameters_course_link = {
                "courseCode": row["courseCode"],
                "groupId": group_id
            }

            try:
                neo4j_graph.query(cypher_course_link, parameters_course_link)
                logging.info(f"Unlinked Course {row['courseCode']} from PrerequisiteGroup {group_id}")
            except Exception as e:
                logging.error(f"Error unlinking Course {row['courseCode']} from PrerequisiteGroup {group_id}: {e}")

            # Optionally, delete the PrerequisiteGroup node if it has no relationships
            cypher_delete_group = """
                                  MATCH (g:PrerequisiteGroup {groupId: $groupId})
                                  WHERE NOT (g)--()
                                  DELETE g
                                  """
            parameters_delete_group = {
                "groupId": group_id
            }

            try:
                neo4j_graph.query(cypher_delete_group, parameters_delete_group)
                logging.info(f"Deleted PrerequisiteGroup {group_id} as it has no remaining relationships")
            except Exception as e:
                logging.error(f"Error deleting PrerequisiteGroup {group_id}: {e}")


In [ ]:
unlink_prerequisite(kg, "data/pre_requisite_relationship.csv")

In [75]:
# This is to find the course pre requisite with OR logic

cypher = """
         MATCH (c:Course {courseCode: 'SC2002'})
        -[:PRE_REQUISITE_FOR]->(g)
        -[:REQUIRES]->(pre)
        RETURN pre.courseCode AS PrerequisiteCourseCode, pre.title AS PrerequisiteTitle
         """
results = kg.query(cypher)
results


[{'PrerequisiteCourseCode': 'SC1003',
  'PrerequisiteTitle': 'Introduction to Computational Thinking and Programming'},
 {'PrerequisiteCourseCode': 'SC1007',
  'PrerequisiteTitle': 'Data Structures & Algorithm'}]

In [77]:
cypher = """
         MATCH (y:yearStanding)
        -[:PRE_REQUISITE_FOR]->(c:Course {courseCode: 'SC3010'})
        RETURN y
         """
results = kg.query(cypher)
results


[{'y': {'year': 'Year 3'}}]

In [78]:
# Add the co_requisite relationship 
def link_corequisite(neo4j_graph, filepath):
    df = pd.read_csv(filepath)

    for index, row in df.iterrows():
        if row["corequisiteType"] == "Course":
            cypher_query = """
                           MATCH (pre: Course {courseCode: $corequisiteValue})
                           MATCH (c: Course {courseCode: $courseCode})
                           MERGE (pre)-[:CO_REQUISITE_FOR]->(c)
                           """
            parameters = {
               "corequisiteValue": row["corequisiteValue"],
               "courseCode": row["courseCode"]
            }

            try:
                neo4j_graph.query(cypher_query, parameters)
                logging.info(f"Linked Corequisite Course {row['corequisiteValue']} to Course {row['courseCode']}")
            except Exception as e:
                logging.error(f"Error linking Corequisite Course {row['corequisiteValue']} to Course {row['courseCode']}: {e}")


In [79]:
link_corequisite(kg, "data/co_requisite_relationships.csv")

In [82]:
# Add the offered_by relationship between courseCode and degree
def link_degree_course(neo4j_graph, filepath):
    df = pd.read_csv(filepath)

    for index, row in df.iterrows():
        cypher_query = """
                        MATCH (c: Course {courseCode: $courseCode})
                        MATCH (d: Degree {degreeCode: $degreeCode})
                        MERGE (c)-[:OFFERED_BY]->(d)
                        """
        parameters = {
            "courseCode": row["courseCode"],
            "degreeCode": row["degreeCode"]
        }

        try:
            neo4j_graph.query(cypher_query, parameters)
            logging.info(f"Linked Course {row['courseCode']} to Degree {row['degreeCode']}")
        except Exception as e:
            logging.error(f"Error linking Course {row['courseCode']} to Degree {row['degreeCode']}: {e}")


In [83]:
link_degree_course(kg, "data/offered_by_relationship.csv")

In [87]:
# Add the has_type relationship between course and type
def link_course_type(neo4j_graph, filepath):
    df = pd.read_csv(filepath)

    for index, row in df.iterrows():
        cypher_query = """
                        MATCH (c: Course {courseCode: $courseCode})
                        MATCH (t: Type {typeName: $typeName})
                        MERGE (c)-[:HAS_TYPE]->(t)
                        """
        parameters = {
            "courseCode": row["courseCode"],
            "typeName": row["typeName"]
        }

        try:
            neo4j_graph.query(cypher_query, parameters)
            logging.info(f"Linked Course {row['courseCode']} to type {row['typeName']}")
        except Exception as e:
            logging.error(f"Error linking Course {row['courseCode']} to type {row['typeName']}: {e}")


In [88]:
link_course_type(kg, "data/has_type_relationship.csv")

In [89]:
# Add the offers_degree relationship between degree and school
def link_degree_school(neo4j_graph, filepath):
    df = pd.read_csv(filepath)

    for index, row in df.iterrows():
        cypher_query = """
                        MATCH (d: Degree {degreeCode: $degreeCode})
                        MATCH (s: School {schoolCode: $schoolCode})
                        MERGE (s)-[:OFFERS_DEGREE]->(d)
                        """
        parameters = {
            "degreeCode": row["degreeCode"],
            "schoolCode": row["schoolCode"]
        }

        try:
            neo4j_graph.query(cypher_query, parameters)
            logging.info(f"Linked School {row['schoolCode']} to degree {row['degreeCode']}")
        except Exception as e:
            logging.error(f"Error linking School {row['schoolCode']} to degree {row['degreeCode']}: {e}")


In [90]:
link_degree_school(kg, "data/offers_degree_relationship.csv")

#### Scheduled IN 
- A single course can be scheduled in multiple semesters across different degree programs.
- Different degrees may have unique scheduling requirements for the same course.
- The system should flexibly manage these relationships and allow for efficient querying. 
To achieve this, the graph schema must accurately represent the many-to-many relationships between degrees, courses, and semesters. 

##### Proposed Graph Schema
To effectively model the complex scheduling relationships, we will introduce an intermediate node called `ScheduledOffering`. This node serves as a bridge between Degrees, Courses, and Semesters, encapsulating the scheduling details. 

In [105]:
def link_course_to_degree(neo4j_graph, filepath):
    df = pd.read_csv(filepath)
    for index, row in df.iterrows():
        # Create a ScheduledOffering node
        offering_id = f"{row['degreeCode']}_{row['semester']}"
        cypher_offering = """
        MERGE (so:ScheduledOffering {offeringId: $offeringId})
        """
        parameters_offering = {
            "offeringId": offering_id
        }
        try:
            neo4j_graph.query(cypher_offering, parameters_offering)
            logging.info(f"Created/Found ScheduledOffering: {offering_id}")
        except Exception as e:
            logging.error(f"Error creating ScheduledOffering {offering_id}: {e}")

        # Link Degree to ScheduledOffering
        cypher_degree_link = """
                             MATCH (d:Degree {degreeCode: $degreeCode}), (so: ScheduledOffering {offeringId: $offeringId})
                             MERGE (d)-[:HAS_SCHEDULED_OFFERING]->(so)
                             """
        
        parameters_degree_link = {
            "degreeCode": row["degreeCode"],
            "offeringId": offering_id
        }
        try:
            neo4j_graph.query(cypher_degree_link, parameters_degree_link)
            logging.info(f"Linked Degree {row['degreeCode']} to ScheduledOffering {offering_id}")
        except Exception as e:
            logging.error(f"Error linking Degree {row['degreeCode']} to ScheduledOffering {offering_id}: {e}")

        # Link the course to the ScheduleOffering
        cypher_course_link = """
                             MATCH (so: ScheduledOffering {offeringId: $offeringId}), (c: Course {courseCode: $courseCode})
                             MERGE (so)-[:INCLUDES]-(c)
                             """
        parameters_course_link = {
            "courseCode": row["courseCode"],
            "offeringId": offering_id
        }
        try:
            neo4j_graph.query(cypher_course_link, parameters_course_link)
            logging.info(f"Linked Course {row['courseCode']} to ScheduledOffering {offering_id}")
        except Exception as e:
            logging.error(f"Error linking Course {row['courseCode']} to ScheduledOffering {offering_id}: {e}")

        

In [106]:
link_course_to_degree(kg, "data/scheduled_offering_relationship.csv")

In [108]:
cypher_query = """ 
               MATCH (d:Degree {degreeCode: "CSC"})-[:HAS_SCHEDULED_OFFERING]->(so:ScheduledOffering)-[:INCLUDES]-(c:Course)
               RETURN c
               """

result = kg.query(cypher_query)
print(result)

[{'c': {'courseCode': 'SC1003', 'academicUnits': 3, 'description': 'Computational thinking (CT) is the process of analyzing a problem then designing and expressing its solution in such a way that a computer can effectively carry it out. This course aims to take students with no prior experience of thinking in a computational manner to a point where they can derive simple algorithms and code programs to solve basic problems in their domain of studies. Topics also include basic program constructs, simple data structures, and an appreciation of the internal operations of a processor.', 'title': 'Introduction to Computational Thinking and Programming'}}, {'c': {'courseCode': 'SC1004', 'academicUnits': 4, 'description': 'This course aims to support you to learn mathematical concepts related to linear algebra and complex numbers. You will develop set of mathematical skills for applications in computer science and engineering, e.g., machine learning, computer graphics, data science etc.', 'ti

In [109]:
def link_course_to_specialisation(neo4j_graph, filepath):
    df = pd.read_csv(filepath)
    for index, row in df.iterrows():
        # Create a SpecialisationTrack node
        specialisation_id = f"{row['degreeCode']}_{row['specialisationName']}"
        cypher_offering = """
        MERGE (st:SpecialisationTrack {specialisation_id: $specialisation_id})
        """
        parameters_offering = {
            "specialisation_id": specialisation_id
        }
        try:
            neo4j_graph.query(cypher_offering, parameters_offering)
            logging.info(f"Created/Found SpecialisationTrack: {specialisation_id}")
        except Exception as e:
            logging.error(f"Error creating SpecialisationTrack {specialisation_id}: {e}")

        # Link Degree to SpecialisationTrack
        cypher_degree_link = """
                             MATCH (d:Degree {degreeCode: $degreeCode}), (st: SpecialisationTrack {specialisation_id: $specialisation_id})
                             MERGE (d)-[:HAS_SPECIALISATION_TRACK]->(st)
                             """
        
        parameters_degree_link = {
            "degreeCode": row["degreeCode"],
            "specialisation_id": specialisation_id
        }
        try:
            neo4j_graph.query(cypher_degree_link, parameters_degree_link)
            logging.info(f"Linked Degree {row['degreeCode']} to SpecialisationTrack {specialisation_id}")
        except Exception as e:
            logging.error(f"Error linking Degree {row['degreeCode']} to SpecialisationTrack {specialisation_id}: {e}")

        # Link the course to the ScheduleOffering
        cypher_course_link = """
                             MATCH (st: SpecialisationTrack {specialisation_id: $specialisation_id}), (c: Course {courseCode: $courseCode})
                             MERGE (st)-[:CONTAINS]-(c)
                             """
        parameters_course_link = {
            "courseCode": row["courseCode"],
            "specialisation_id": specialisation_id
        }
        try:
            neo4j_graph.query(cypher_course_link, parameters_course_link)
            logging.info(f"Linked Course {row['courseCode']} to SpecialisationTrack {specialisation_id}")
        except Exception as e:
            logging.error(f"Error linking Course {row['courseCode']} to SpecialisationTrack {specialisation_id}: {e}")

        

In [110]:
link_course_to_specialisation(kg, "data/specialisation_courses_relationship.csv")

In [111]:
cypher_query = """ 
               MATCH (d:Degree {degreeCode: "CSC"})-[:HAS_SPECIALISATION_TRACK]->(st:SpecialisationTrack)-[:CONTAINS]-(c:Course)
               RETURN c
               """

result = kg.query(cypher_query)
print(result)

[{'c': {'courseCode': 'SC3000', 'academicUnits': 3, 'description': 'Computer / software engineers are involved in effective and efficient building of knowledge agent systems that satisfy the requirements of users; possibly software for intelligent embedded and intelligent information systems. General awareness of theory, knowledge, and practice in all phases of the knowledge based systems and representation techniques for problem solving are necessary for those of you who wants to get into the field artificial intelligence. These are advanced intelligent systems that are finding widespread applications in finance, banking, manufacturing industries.', 'title': 'Artificial Intelligence'}}, {'c': {'courseCode': 'SC4000', 'academicUnits': 3, 'description': 'This course introduces basic concepts and methodologies for machine learning as well as their applications. Clustering. Dimension Reduction. Classification. Decision Theory. Density Estimation. Classifier Evaluation.', 'title': 'Machine